In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

#Classificadores Lineares
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

#Classificadores KNN
from sklearn.neighbors import KNeighborsClassifier

#Classificadores Naive Nayes
from sklearn.naive_bayes import MultinomialNB

#Classificadores Arvores de Decisão
from sklearn.tree import DecisionTreeClassifier

#SVM
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import math

from sklearn import linear_model
from scipy.special import expit

from sklearn.metrics import confusion_matrix
import scipy
from scipy.io import arff

import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from sklearn.svm import SVC
#from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.externals import joblib

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut

C:\Users\Vitor Bezerra\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Agenda
### Pipeline
* Definir um workflow de análise
* Criar um workflow que conecta transformadores e estimadores em uma sequência. 
* Cada etapa é realizada por uma classe, que são encapsuladas em uma classe maior e executadas sob controle de um pipeline
### Salvando e Carregando modelos para arquivos
* joblib
### Busca por hiperparametros
* Grid Search
* Random Search
### Validação Cruzada
* Kfolds
* Kfolds estratificados
* LOOCV
* repetição de divisão entre treino e teste
### Escolha de características (Features)
* kbest

### No código abaixo está sendo feita a comparação entre 3 modelos de classificação

* Preparação dos dados para classificação

In [208]:
heart = pd.read_csv('SAheart.data')

X = df.drop(['chd'], axis=1)
y = df['chd']

le = preprocessing.LabelEncoder()
X['famhist'] = le.fit_transform(X['famhist'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [203]:
heart
X = df.drop(['tobacco','typea'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

* Comparação do score dos modelos

In [197]:
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print(sgd.score(X_test, y_test))

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(tree.score(X_test, y_test))

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(knn.score(X_test, y_test))

0.6808510638297872
0.5319148936170213
0.6595744680851063


* Código complexo e dificil de manter

* Pipelines auxiliam sistematizar a avaliação

### Definindo um workflow com pipeline
* A mesma análise anterior, porem, incluindo cada avaliação em um objeto pipe
* Ao final um pipeline com outros pipes sao executados
* Um pipe possui uma ou mais transformacoes,por exemplo, normalização, padronizaçõ, etc (que possue a funcao transform),  e por último um modelo de predição (que possui a função fit())

In [198]:
# Construindo pipelines
pipe_lr = Pipeline( [ ('scl', StandardScaler()), ('clf', LogisticRegression()) ] )
pipe_knn = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipe_dt = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

# Lista de pipelines a serem executados
pipelines = [pipe_lr, pipe_knn, pipe_dt]

# Dicionário para facilitar identificacao
pipe_dict = {0: 'Logistic Regression', 1: 'KNN', 2: 'Decision Tree'}

# aplicando fit
# Generaliza a execucao do fit de cada ultima funcao do pipe
for pipe in pipelines:
    pipe.fit(X_train, y_train)

# Compara acurácia
for idx, val in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

# para cada modelo treinado obtem val score
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
    # Descobre o melhor val.score e armazen em best_clf
    if val.score(X_test, y_test) > best_acc:
        best_acc = val.score(X_test, y_test)
        best_pipe = val
        best_clf = idx
print('Classifier with best accuracy: %s' % pipe_dict[best_clf])

Logistic Regression pipeline test accuracy: 0.702
KNN pipeline test accuracy: 0.660
Decision Tree pipeline test accuracy: 0.511
Classifier with best accuracy: Logistic Regression


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Salvando um modelo para um arquivo usando joblib

from sklearn.externals import joblib

In [199]:
# Save pipeline to file
joblib.dump(best_pipe, 'best_pipeline.pkl', compress=1)

['best_pipeline.pkl']

#### Carregando um modelo salvo em arquivo

In [209]:
joblib_model= joblib.load('best_pipeline.pkl')

y_pred = joblib_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[26  5]
 [10  6]]


### Busca por hiperparametros
* Hiperparametros sao as diversas parametrizações possíveis de um modelo de predição
* Usando gridsearchCV: busca todas as combinações definidas de hiperparametros, e retorna combinação com o melhor score

In [210]:
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [1, 10, 100, 1000]},]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    clf = GridSearchCV(
        RidgeClassifier(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("classification report:")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'alpha': 100}

Grid scores on development set:

0.692 (+/-0.103) for {'alpha': 1}
0.698 (+/-0.095) for {'alpha': 10}
0.705 (+/-0.108) for {'alpha': 100}
0.656 (+/-0.051) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.68      0.84      0.75        31
           1       0.44      0.25      0.32        16

    accuracy                           0.64        47
   macro avg       0.56      0.54      0.54        47
weighted avg       0.60      0.64      0.61        47


# Tuning hyper-parameters for recall



/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'alpha': 10}

Grid scores on development set:

0.675 (+/-0.066) for {'alpha': 1}
0.681 (+/-0.059) for {'alpha': 10}
0.676 (+/-0.044) for {'alpha': 100}
0.624 (+/-0.024) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.68      0.87      0.76        31
           1       0.43      0.19      0.26        16

    accuracy                           0.64        47
   macro avg       0.55      0.53      0.51        47
weighted avg       0.59      0.64      0.59        47




* Usando RandomizedSearchCV: busca um conjunto aleatório de combinações possíveis de hiperparametros, e retorna o modelo com o melhor score

In [212]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200, random_state=0)

distributions = dict(penalty=['l2', 'l1'])

clf = RandomizedSearchCV(logistic, distributions, random_state=0)

search = clf.fit(X_test, y_test)

search.best_params_


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'penalty': 'l2'}

### Validação Cruzada
#### K-fold
* fold: sub conjunto de dados para teste e treino
* Sub conjuntos gerados movimentando um índice nos dados para definir o início e fim dos dados de treino
* número K define quantidade de movimentações para gerar base de teste

![Folds](https://www.researchgate.net/profile/Mingchao_Li/publication/331209203/figure/fig2/AS:728070977748994@1550597056956/K-fold-cross-validation-method.png)

### Exemplo de Kfold iterando pelos índices

In [217]:
# Amostra de dados
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
# Montando Folds
kfold = KFold(5)


for train, test in kfold.split(data):
    #print(train , test)
    print(data[train] , data[test])

[0.3 0.4 0.5 0.6 0.7 0.8] [0.1 0.2]
[0.1 0.2 0.5 0.6 0.7 0.8] [0.3 0.4]
[0.1 0.2 0.3 0.4 0.7 0.8] [0.5 0.6]
[0.1 0.2 0.3 0.4 0.5 0.6 0.8] [0.7]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7] [0.8]


### Comparando acurácia de classificação com validação cruzada


In [229]:
heart = pd.read_csv('SAheart.data')

le = preprocessing.LabelEncoder()
heart['famhist_label'] = le.fit_transform(heart['famhist'])
heart.drop('famhist', axis=1, inplace=True)

print(heart.head())


X = heart[['sbp','tobacco','ldl','adiposity','typea','obesity','alcohol','age','famhist_label']] 
y = heart[['chd']] 

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.15)

   sbp  tobacco   ldl  adiposity  typea  obesity  alcohol  age  chd  \
1  160    12.00  5.73      23.11     49    25.30    97.20   52    1   
2  144     0.01  4.41      28.61     55    28.87     2.06   63    1   
3  118     0.08  3.48      32.28     52    29.14     3.81   46    0   
4  170     7.50  6.41      38.03     51    31.99    24.26   58    1   
5  134    13.60  3.50      27.78     60    25.99    57.34   49    1   

   famhist_label  
1              1  
2              0  
3              1  
4              1  
5              1  


In [230]:
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print("Acuracia: %.2f%%" % (sgd.score(X_test, y_test)*100.0)) 

Acuracia: 54.29%


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Teste com Kfolds
* Montando 10 folds para o mesmo classificador testado acima
* cross_val_score aplica os folds e obtem o score do fit de cada sub conjunto
* Ao final mostra a média das acurácias obtidas

In [232]:
kfold = KFold(n_splits=10, random_state=100)
model_kfold = SGDClassifier()
results_kfold = cross_val_score(model_kfold, X_train, y_train, cv=kfold)

print("scores: ", results_kfold) 

print("Acuracia: %.2f%%" % (results_kfold.mean()*100.0)) 

scores:  [0.475      0.575      0.51282051 0.41025641 0.51282051 0.66666667
 0.71794872 0.53846154 0.17948718 0.43589744]
Acuracia: 50.24%


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed w

### Folds Stratificados

* Utiliza folds estratificados: cada conjunto contendo aproximadamente a mesma proporção de labels de destino que os dados completos.


In [234]:
skfold = StratifiedKFold(n_splits=4, random_state=100)
model_skfold = SGDClassifier()
results_skfold = cross_val_score(model_skfold, X_train, y_train, cv=skfold)
print("scores: ", results_skfold) 
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

scores:  [0.66666667 0.57142857 0.33673469 0.69072165]
Accuracy: 56.64%


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed w

### Leave One Out Cross-Validation (LOOCV)

* Os Fold são definidos com tamanho 1 e K o número de observações

* Essa variação é útil quando os dados de treinamento são de tamanho limitado e o número de parâmetros a serem testados não é alto.

In [236]:
loocv = LeaveOneOut()
model_loocv = SGDClassifier()
results_loocv = cross_val_score(model_loocv, X_train, y_train, cv=loocv)
#print("scores: ", results_loocv) 
print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed w

Accuracy: 60.71%


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Repetição Aleatória de divisão entre treino e teste (Repeated Random Test-Train Splits)

* Híbrido entre divisão tradicional de teste de trem e do método de validação cruzada de k Fold. 

* Nesta técnica, divisões entre treino e teste aleatórias são criadas nos dados da maneira definida pelo conjunto de testes de treinamento 

* Esse processo é repetidos várias vezes, assim como o método de validação cruzada.

In [237]:
kfold2 = ShuffleSplit(n_splits=10, test_size=0.30, random_state=100)
model_shufflecv = SGDClassifier()
results_4 = cross_val_score(model_shufflecv, X_train, y_train, cv=kfold2)
print("scores: ", results_4) 
print("Accuracy: %.2f%% (%.2f%%)" % (results_4.mean()*100.0, results_4.std()*100.0))

scores:  [0.40677966 0.3220339  0.6440678  0.75423729 0.68644068 0.39830508
 0.61864407 0.60169492 0.68644068 0.61016949]
Accuracy: 57.29% (13.75%)


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed w

### Combinando PIpeline e Grid Search

In [239]:
#pipe_knn = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipetree = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

#pipe = [pipe_knn, pipetre]
pipe = [pipetree]

param_range = [1, 2, 3, 4, 5]

# grid search params
#grid_params = [{'clf__criterion': ['gini', 'entropy'],
#               'clf__presort': [True, False]}]
grid_params = [{'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': param_range,
    'clf__max_depth': param_range,
    'clf__min_samples_split': param_range[1:],
    'clf__presort': [True, False]}]

# Construct grid search
gs = GridSearchCV(estimator=pipetree,
    param_grid=grid_params,
    scoring='accuracy')

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best accuracy: 0.691

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 5, 'clf__min_samples_split': 2, 'clf__presort': False}


### Escolhendo Features 
* Método kbest
* Utiliza método chi quadrado para escolher melhor conjunto de features de acordo com o seguinte critério:
* Realiza um teste estatístico usando qui-quadrado entre cada features e classe
* O teste do qui-quadrado “elimina” features com maior probabilidade de serem independentes da classe e, portanto, irrelevantes para a classificação.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2

In [246]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = load_iris(return_X_y=True)
print(X.shape)
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print(X_new.shape)
print(X_new)

(150, 4)
(150, 2)
[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [1.7 0.4]
 [1.4 0.3]
 [1.5 0.2]
 [1.4 0.2]
 [1.5 0.1]
 [1.5 0.2]
 [1.6 0.2]
 [1.4 0.1]
 [1.1 0.1]
 [1.2 0.2]
 [1.5 0.4]
 [1.3 0.4]
 [1.4 0.3]
 [1.7 0.3]
 [1.5 0.3]
 [1.7 0.2]
 [1.5 0.4]
 [1.  0.2]
 [1.7 0.5]
 [1.9 0.2]
 [1.6 0.2]
 [1.6 0.4]
 [1.5 0.2]
 [1.4 0.2]
 [1.6 0.2]
 [1.6 0.2]
 [1.5 0.4]
 [1.5 0.1]
 [1.4 0.2]
 [1.5 0.2]
 [1.2 0.2]
 [1.3 0.2]
 [1.4 0.1]
 [1.3 0.2]
 [1.5 0.2]
 [1.3 0.3]
 [1.3 0.3]
 [1.3 0.2]
 [1.6 0.6]
 [1.9 0.4]
 [1.4 0.3]
 [1.6 0.2]
 [1.4 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [4.7 1.4]
 [4.5 1.5]
 [4.9 1.5]
 [4.  1.3]
 [4.6 1.5]
 [4.5 1.3]
 [4.7 1.6]
 [3.3 1. ]
 [4.6 1.3]
 [3.9 1.4]
 [3.5 1. ]
 [4.2 1.5]
 [4.  1. ]
 [4.7 1.4]
 [3.6 1.3]
 [4.4 1.4]
 [4.5 1.5]
 [4.1 1. ]
 [4.5 1.5]
 [3.9 1.1]
 [4.8 1.8]
 [4.  1.3]
 [4.9 1.5]
 [4.7 1.2]
 [4.3 1.3]
 [4.4 1.4]
 [4.8 1.4]
 [5.  1.7]
 [4.5 1.5]
 [3.5 1. ]
 [3.8 1.1]
 [3.7 1. ]
 [3.9 1.2]
 [5.1 1.6]
 [4.5 1.5]
 [4.5 1.6]
 [4.7 1.5]
 [4.4 1.3]
 [4.1 1.3]
 [4

In [250]:
heart = pd.read_csv('SAheart.data')

le = preprocessing.LabelEncoder()
heart['famhist_label'] = le.fit_transform(heart['famhist'])
heart.drop('famhist', axis=1, inplace=True)

#print(heart.head())


X = heart[['sbp','tobacco','ldl','adiposity','typea','obesity','alcohol','age','famhist_label']] 
y = heart[['chd']] 

print(X)
X_new = SelectKBest(chi2, k=3).fit_transform(X, y)
print(X_new)


     sbp  tobacco    ldl  adiposity  typea  obesity  alcohol  age  \
1    160    12.00   5.73      23.11     49    25.30    97.20   52   
2    144     0.01   4.41      28.61     55    28.87     2.06   63   
3    118     0.08   3.48      32.28     52    29.14     3.81   46   
4    170     7.50   6.41      38.03     51    31.99    24.26   58   
5    134    13.60   3.50      27.78     60    25.99    57.34   49   
..   ...      ...    ...        ...    ...      ...      ...  ...   
459  214     0.40   5.98      31.72     64    28.45     0.00   58   
460  182     4.20   4.41      32.10     52    28.61    18.72   52   
461  108     3.00   1.59      15.23     40    20.09    26.64   55   
462  118     5.40  11.61      30.79     64    27.35    23.97   40   
463  132     0.00   4.82      33.41     62    14.70     0.00   46   

     famhist_label  
1                1  
2                0  
3                1  
4                1  
5                1  
..             ...  
459              0  
460

### Exercicio

#### Montem uma classificacao utilizando a base da Veltec ou Senai
#### Monte testes de avaliacao de diferentes classificadores considerando:
* Busca por hiperparametros (Considere testar parametros de regularizacao)
* Busca por features
* Utilize um metodo de validacao cruzada

In [2]:
import pandas as pd
vigencias = pd.read_csv('vigencias_scores.csv')
vigencias.head()

,id_vei,id_uo_vei,id_motorista,id_uo_motorista,distancia_percorrida_km,qtd_banguela,qtd_curvas,qtd_aceleracoes,qtd_frenagens,qtd_vel_faixa_1,...,qtd_vel_via_faixa_3,tempo_vel_via_faixa_1,tempo_vel_via_faixa_2,tempo_vel_via_faixa_3,qtd_manuseio_celular,qtd_fadiga_motorista,qtd_distracao_motorista,qtd_uso_cigarro,score_geral,rank
0,66486,2855,636779.0,2950.0,234.420,0.0,0.0,0.0,0.0,0.0,...,1.0,208.0,0.0,1456.0,0.0,0.0,0.0,0.0,50.000000,ruim
1,8155,3321,636786.0,2950.0,114.083,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.962685,regular
2,8577,3327,636592.0,2950.0,183.725,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,otimo
3,7665,3581,636644.0,2950.0,702.525,0.0,0.0,0.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.437814,otimo
4,8427,2868,636989.0,2950.0,175.290,0.0,0.0,1.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.308403,bom


## Busca por features

In [4]:
X = vigencias.drop(['rank'], axis=1)
y = vigencias['rank']

X.head()

,id_vei,id_uo_vei,id_motorista,id_uo_motorista,distancia_percorrida_km,qtd_banguela,qtd_curvas,qtd_aceleracoes,qtd_frenagens,qtd_vel_faixa_1,...,qtd_vel_via_faixa_2,qtd_vel_via_faixa_3,tempo_vel_via_faixa_1,tempo_vel_via_faixa_2,tempo_vel_via_faixa_3,qtd_manuseio_celular,qtd_fadiga_motorista,qtd_distracao_motorista,qtd_uso_cigarro,score_geral
0,66486,2855,636779.0,2950.0,234.420,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,208.0,0.0,1456.0,0.0,0.0,0.0,0.0,50.000000
1,8155,3321,636786.0,2950.0,114.083,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.962685
2,8577,3327,636592.0,2950.0,183.725,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000
3,7665,3581,636644.0,2950.0,702.525,0.0,0.0,0.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.437814
4,8427,2868,636989.0,2950.0,175.290,0.0,0.0,1.0,3.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.308403


In [35]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_new = SelectKBest(chi2, k=12).fit_transform(X, y)
print(X_new.shape)
print(X_new)
X_new

(12642, 12)
[[6.64860000e+04 2.85500000e+03 6.36779000e+05 ... 0.00000000e+00
  1.45600000e+03 5.00000000e+01]
 [8.15500000e+03 3.32100000e+03 6.36786000e+05 ... 0.00000000e+00
  0.00000000e+00 7.89626851e+01]
 [8.57700000e+03 3.32700000e+03 6.36592000e+05 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+02]
 ...
 [2.44030000e+04 2.87000000e+03 6.36570000e+05 ... 0.00000000e+00
  5.21300000e+03 5.00000000e+01]
 [9.85300000e+03 2.86800000e+03 6.36811000e+05 ... 0.00000000e+00
  0.00000000e+00 9.33547458e+01]
 [1.06130000e+04 2.87100000e+03 6.36636000e+05 ... 0.00000000e+00
  0.00000000e+00 9.80148500e+01]]


array([[6.64860000e+04, 2.85500000e+03, 6.36779000e+05, ...,
        0.00000000e+00, 1.45600000e+03, 5.00000000e+01],
       [8.15500000e+03, 3.32100000e+03, 6.36786000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 7.89626851e+01],
       [8.57700000e+03, 3.32700000e+03, 6.36592000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+02],
       ...,
       [2.44030000e+04, 2.87000000e+03, 6.36570000e+05, ...,
        0.00000000e+00, 5.21300000e+03, 5.00000000e+01],
       [9.85300000e+03, 2.86800000e+03, 6.36811000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 9.33547458e+01],
       [1.06130000e+04, 2.87100000e+03, 6.36636000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 9.80148500e+01]])

## Busca por hiperparametros (Considere testar parametros de regularizacao)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size=0.1)

In [37]:
pipetree = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

#pipe = [pipe_knn, pipetre]
pipe = [pipetree]

param_range = [1, 2, 3, 4, 5]

# grid search params
#grid_params = [{'clf__criterion': ['gini', 'entropy'],
#               'clf__presort': [True, False]}]
grid_params = [{'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': param_range,
    'clf__max_depth': param_range,
    'clf__min_samples_split': param_range[1:],
    'clf__presort': [True, False]}]

# Construct grid search
gs = GridSearchCV(estimator=pipetree,
    param_grid=grid_params,
    scoring='accuracy', n_jobs=-1)

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

C:\Users\Vitor Bezerra\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best accuracy: 0.999

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__presort': True}


## Utilize um metodo de validacao cruzada

In [38]:
kfold = KFold(n_splits=10, random_state=100)
model_kfold = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=1, min_samples_split=2, presort=True)
results_kfold = cross_val_score(model_kfold, X_train, y_train, cv=kfold)

print("scores: ", results_kfold) 

print("Acuracia: %.2f%%" % (results_kfold.mean()*100.0)) 

scores:  [1.         1.         0.99912127 1.         1.         0.99824253
 0.99912127 1.         1.         1.        ]
Acuracia: 99.96%


In [39]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=1, min_samples_split=2, presort=True)
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bom       1.00      1.00      1.00        98
       otimo       1.00      1.00      1.00       760
     pessimo       1.00      1.00      1.00        62
     regular       1.00      1.00      1.00       101
        ruim       1.00      1.00      1.00       244

    accuracy                           1.00      1265
   macro avg       1.00      1.00      1.00      1265
weighted avg       1.00      1.00      1.00      1265



In [40]:
senai = pd.read_csv('senai_inep.csv')
senai.head()

,CO_IES,NO_IES,CO_CURSO,NO_CURSO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,ID_ALUNO,TP_COR_RACA,TP_SEXO,...,IN_DEFICIENCIA,DT_INGRESSO_CURSO,IN_RESERVA_VAGAS,IN_FINANCIAMENTO_ESTUDANTIL,IN_APOIO_SOCIAL,IN_ATIVIDADE_EXTRACURRICULAR,TP_ESCOLA_CONCLUSAO_ENS_MEDIO,IN_MATRICULA,IN_CONCLUINTE,TP_SITUACAO
0,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,C2E870F652B2299ED62D4AA8038A9422,1,1,...,0,01JUL2009:00:00:00,0,0.0,0,0,1,0,0,0
1,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,81A4D85417E7123C8DF075A0A2629E8E,0,1,...,0,01JUL2009:00:00:00,0,0.0,0,0,1,1,1,1
2,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,77F900FA4B592D61029032DCA66D9F55,2,2,...,0,01JAN2014:00:00:00,0,0.0,0,0,1,1,1,1
3,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,4D21A9481680ED505C359CE9A0F8F212,0,2,...,0,01JAN2006:00:00:00,0,0.0,0,0,9,0,0,1
4,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,63341AACBCB526FE3A929666DE581731,0,1,...,0,01JAN2010:00:00:00,0,0.0,0,0,1,0,0,0


In [100]:
X_1 = senai.loc[senai['TP_SITUACAO'] == 1, ]
X_0 = senai.loc[senai['TP_SITUACAO'] == 0, ]
X_1 = X_1.iloc[0:427,]
frames = [X_0,X_1]
X_3 = pd.concat(frames)
X = X_3.drop(['TP_SITUACAO', 'IN_CONCLUINTE', 'ID_ALUNO', 'DT_INGRESSO_CURSO'], axis=1)

y = X_3['TP_SITUACAO']
le = preprocessing.LabelEncoder()
for col in ['NO_IES', 'NO_CURSO']:
    X[col] = le.fit_transform(X[col])
print(X.shape)
X.head()

(854, 18)


,CO_IES,NO_IES,CO_CURSO,NO_CURSO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_NACIONALIDADE,IN_DEFICIENCIA,IN_RESERVA_VAGAS,IN_FINANCIAMENTO_ESTUDANTIL,IN_APOIO_SOCIAL,IN_ATIVIDADE_EXTRACURRICULAR,TP_ESCOLA_CONCLUSAO_ENS_MEDIO,IN_MATRICULA
0,1400,0,20516,0,3.0,1.0,1,1,1,30,1,0,0,0.0,0,0,1,0
4,1400,0,20516,0,3.0,1.0,1,0,1,31,1,0,0,0.0,0,0,1,0
10,1400,0,20516,0,3.0,1.0,1,0,2,29,1,0,0,0.0,0,0,2,0
11,1400,0,20516,0,3.0,1.0,1,1,1,37,1,0,0,0.0,0,0,9,0
14,1400,0,20516,0,3.0,1.0,1,1,1,29,1,0,0,0.0,0,0,1,0


In [70]:
senai.describe()

,CO_IES,CO_CURSO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_NACIONALIDADE,IN_DEFICIENCIA,IN_RESERVA_VAGAS,IN_FINANCIAMENTO_ESTUDANTIL,IN_APOIO_SOCIAL,IN_ATIVIDADE_EXTRACURRICULAR,TP_ESCOLA_CONCLUSAO_ENS_MEDIO,IN_MATRICULA,IN_CONCLUINTE,TP_SITUACAO
count,3309.000000,3.309000e+03,3309.0,3309.000000,3309.0,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.0,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000
mean,5784.009368,9.181798e+05,3.0,2.054397,1.0,1.288305,1.583258,26.365367,1.002418,1.377455,0.0,0.004533,0.011484,0.010275,1.403747,0.703234,0.102750,0.870958
std,6060.657373,1.333823e+06,0.0,0.964642,0.0,0.892830,0.493094,7.675147,0.069505,3.235962,0.0,0.067186,0.106562,0.100859,1.128475,0.456902,0.303678,0.335297
min,1400.000000,2.051600e+04,3.0,1.000000,1.0,0.000000,1.000000,17.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1400.000000,6.844000e+04,3.0,1.000000,1.0,1.000000,1.000000,21.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,1400.000000,1.060125e+06,3.0,2.000000,1.0,1.000000,2.000000,24.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,13677.000000,1.110546e+06,3.0,3.000000,1.0,1.000000,2.000000,30.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,1.000000
max,15445.000000,5.001208e+06,3.0,3.000000,1.0,5.000000,2.000000,80.000000,3.000000,9.000000,0.0,1.000000,1.000000,1.000000,9.000000,1.000000,1.000000,1.000000


In [101]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_new = SelectKBest(chi2, k=9).fit_transform(X, y)
print(X_new.shape)
print(X_new)


X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size=0.1)



(854, 9)
[[1.4000e+03 0.0000e+00 2.0516e+04 ... 3.0000e+01 0.0000e+00 0.0000e+00]
 [1.4000e+03 0.0000e+00 2.0516e+04 ... 3.1000e+01 0.0000e+00 0.0000e+00]
 [1.4000e+03 0.0000e+00 2.0516e+04 ... 2.9000e+01 0.0000e+00 0.0000e+00]
 ...
 [1.4000e+03 0.0000e+00 5.9822e+04 ... 2.8000e+01 0.0000e+00 0.0000e+00]
 [1.4000e+03 0.0000e+00 5.9822e+04 ... 2.8000e+01 0.0000e+00 1.0000e+00]
 [1.4000e+03 0.0000e+00 5.9822e+04 ... 2.3000e+01 0.0000e+00 1.0000e+00]]


In [102]:
pipetree = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

#pipe = [pipe_knn, pipetre]
pipe = [pipetree]

param_range = [1, 2, 3, 4, 5]

# grid search params
#grid_params = [{'clf__criterion': ['gini', 'entropy'],
#               'clf__presort': [True, False]}]
grid_params = [{'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': param_range,
    'clf__max_depth': param_range,
    'clf__min_samples_split': param_range[1:],
    'clf__presort': [True, False]}]

# Construct grid search
gs = GridSearchCV(estimator=pipetree,
    param_grid=grid_params,
    scoring='accuracy', n_jobs=-1)

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

C:\Users\Vitor Bezerra\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best accuracy: 0.914

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__presort': True}


In [103]:
kfold = KFold(n_splits=10, random_state=100)
model_kfold = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=5, min_samples_split=2, presort=True)
results_kfold = cross_val_score(model_kfold, X_train, y_train, cv=kfold)

print("scores: ", results_kfold) 

print("Acuracia: %.2f%%" % (results_kfold.mean()*100.0)) 

scores:  [0.94805195 0.8961039  0.83116883 0.92207792 0.97402597 0.87012987
 0.93506494 0.92207792 0.85526316 0.85526316]
Acuracia: 90.09%


In [104]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=1, min_samples_split=5, presort=True)
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.98      1.00      0.99        45

    accuracy                           0.99        86
   macro avg       0.99      0.99      0.99        86
weighted avg       0.99      0.99      0.99        86

